In [1]:
import os
os.chdir("/Users/ruichenyang/projects/ds_jobs/")
os.getcwd()

'/Users/ruichenyang/projects/ds_jobs'

In [12]:

import boto3
s3_client = boto3.client('s3')
import awswrangler as wr
import pandas as pd
import numpy as np

from openai import OpenAI
ai_client = OpenAI()

bucket = "datascience-linkedin-job-scrape"

In [13]:
def list_all_objects(bucket, prefix):
    continuation_token = None
    fpaths = []

    while True:
        kwargs = {
            'Bucket': bucket,
            'Prefix': prefix,
        }
        if continuation_token:
            kwargs.update({'ContinuationToken': continuation_token})
            
        response = s3_client.list_objects_v2(**kwargs)
        if response.get('Contents'):
            for c in response['Contents']:
                fpaths.append(c['Key'])
        
        continuation_token = response.get('NextContinuationToken')

        if continuation_token is None:
            break

    return fpaths

In [14]:
fpaths = list_all_objects(bucket, prefix='data')

In [15]:
ds_jobs = []
for f in fpaths[:1]:
    data = wr.s3.read_csv(path=f"s3://{bucket}/{f}").assign(**{'scrape_dt': lambda x: f.split("_")[-1][:-4]})
    ds_jobs.append(data)

In [16]:
df = pd.concat(ds_jobs, axis=0).loc[lambda x: ~x['company'].isnull()].sort_values(by=['company', 'scrape_dt']).reset_index(drop=True)

In [17]:
df.head()

,company,position,location,full_text,details,post_date,scrape_dt
0,23andMe,"Scientist II, Statistical Genetics","Sunnyvale, CA","Scientist II, Statistical Genetics, 23andMe, S...",About the job\n23andMe is looking for a quanti...,NaN,2023-03-09
1,7-Eleven,Senior UI Developer,"Texas, United States","Senior UI Developer, 7-Eleven, Texas, United S...",About the job\nExpertise in Node.JS developmen...,NaN,2023-03-09
2,ACS Solutions,Bioprocess Associate,"Gaithersburg, MD","Bioprocess Associate, ACS Solutions, Gaithersb...",About the job\nPosition - Associate Scientist/...,NaN,2023-03-09
3,AMDG Architects,Project Coordinator,"Grand Rapids, MI","Project Coordinator, AMDG Architects, Grand Ra...",About the job\n\nAMDG Architects is looking fo...,NaN,2023-03-09
4,ASAPP,Speech Scientist-Language Technology,"Mountain View, CA","Speech Scientist-Language Technology, ASAPP, M...","About the job\nAt ASAPP, we’re on a mission to...",NaN,2023-03-09


In [18]:
print(df['details'].values[0])

About the job
23andMe is looking for a quantitative scientist with extensive experience in statistical analysis of human genetics data to join our research team. You will develop and apply statistical methods in order to derive insights from 23andMe’s large-scale genetic and phenotypic datasets, with a focus on pleiotropy and disease modeling. You will work cross-functionally with research and therapeutics scientists to support target discovery and validation efforts.

With the world’s largest database of more than ten million genotyped customers, 23andMe is at the forefront of using human genetics to advance biomedical research and transform healthcare. Join our growing research team in translating genetic discoveries into insights into molecular biology of human traits and diseases.

Who We Are

Since 2006, 23andMe’s mission has been to help people access, understand, and benefit from the human genome. We are a group of passionate individuals pushing the boundaries of what’s possible

In [19]:
test_jd = df['details'].values[0]

In [20]:
completion = ai_client.chat.completions.create(
    model="gpt-3.5-turbo", 
    messages=[
        {"role": "system", "content": "You are a hiring manager, skilled in summarizing and extracting key business domain information and technical skills out of the text."},
        {"role": "user", "content": f"given the following job description: {test_jd} (the end of the job description), can you extract the technical skills"}
    ]
)

In [21]:
print(completion.choices[0].message)

ChatCompletionMessage(content='- Extensive experience in statistical analysis of human genetics data\n- Development and application of statistical methods\n- Pleiotropy and disease modeling\n- Software development for statistical analysis of large-scale genomic and phenotypic datasets\n- Analyzing genetic variants associated with specific inherited traits and across multiple traits\n- Modeling phenotypes to derive genetic insights and therapeutic hypotheses\n- Working with large datasets in R and Python, with C++ experience as a plus', role='assistant', function_call=None, tool_calls=None)


In [28]:
print(completion.choices[0].message.content)

- Extensive experience in statistical analysis of human genetics data
- Development and application of statistical methods
- Pleiotropy and disease modeling
- Software development for statistical analysis of large-scale genomic and phenotypic datasets
- Analyzing genetic variants associated with specific inherited traits and across multiple traits
- Modeling phenotypes to derive genetic insights and therapeutic hypotheses
- Working with large datasets in R and Python, with C++ experience as a plus
